In [41]:
import numpy as np
import random
from time import time, ctime

## Datasets

### Moreno Health (friendships) 

- Size	2,539 vertices (students)
- Volume	12,969 edges (friendships)
- Wedge count	99,247
- Triangle count	4,694
- Clustering coefficient	14.2%

### Actor collaborations 

- Size: 382,219 vertices (actors)
- Volume: 33,115,812 edges (collaborations)
- Unique volume: 30,076,166 edges (collaborations)
- Average degree: 173.28 edges / vertex
- Wedge count: 6,266,209,411 
- Triangle count	346,813,199 
- Clustering Coefficient (transitivity): 	16.6% 


A common measure is the transitivity = 3T/W

In [42]:
path = "../data/moreno_health/out.moreno_health_health"

# path = "../data/actor-collaboration/out.actor-collaboration"

### Core algorithm

#### Wedge sampling to estimate the transitivity
A wedge is closed if it participates in a triangle, otherwise it is open. Note that the transitivity (= 3T/W) is exactly the probability that a uniform random wedge is closed. This leads to a simple randomized algorithm for estimating transivity (and triangle count) by sampling a set of independent uniform random wedges and finding the fraction of closed wedges. 

#### But how to sample wedges from a stream of edges? 
Suppose we just sampled a uniform random set of edges. How large does this set need to be to get a wedge? The birthday paradox can be used to deduce that (as long as W ≥ m, which holds for a great majority, if not all, of real networks) O(sqrt(n)) edges suffice


#### Reservoir sampling 
Reservoir sampling means sampling a stream sequentially, with probability 1/i to pick the i:th data point. 

#### Main data structures 

- Array edge res[1...se]: This is the array of reservoir edges and is the subsample of the stream maintained.
- New wedges Nt: This is a list of all wedges involving et formed only by edges in edge res. This may often be empty, if et is not added to the edge res. We do not necessarily maintain this list explicitly, and we discuss implementation details later.
- Variable tot wedges: This is the total number of wedges formed by edges in the current edge res.
- Array wedge res[1...sw]: This is an array of reservoir wedges of size sw.
- Array isClosed[1...sw]: This is a boolean array. We set isClosed[i] to be true if wedge wedge res[i] is detected as closed.

#### Algorithm
![update.png](./update.png)

In [46]:
class StreamingTriangles: 
    def __init__(this, path, se, sw, early_stop = np.inf): 
        ## Initialize edge_res of size se and wedge_res of size sw
        this.se = se
        this.sw = sw
        
        this.edge_res = [(0,0)] * se
        this.wedge_res = [((0,0), (0,0))] * sw # containing the sampled wedges, a tuple of two tuples

        this.tot_wedges = 0 # the number of wedges formed by edges in the current edge res
        
        this.isClosed = np.zeros(sw, dtype = bool)
        
        this.K = [0]
        this.T = [0]
        
        this.t = 1 # t denotes the timestep 
        
        
        print("Starting at {}".format(ctime(time())))

        ## Simulate stream
        with open(path, encoding='utf-8', errors='replace') as f:

            f.readline() # skip first line 
            
            while this.t < early_stop or this.t < 33115812: 
                ## read data from line
                line = f.readline()
                
                edge = (int(line.split(" ")[0]), int(line.split(" ")[1]))
                
                ## We want the edge nodes to be ordered
                if (edge[0] > edge[1]):
                    edge = (edge[1], edge[0])
                
                ## Initialize during the first se rounds
                if (this.t < this.se): 
                    this.edge_res[this.t] = edge
                    wedges = list(this.getAllWedges())
                    for i, w in enumerate(wedges): 
                        this.wedge_res[i] = w
                                        
                # call update 
                this.update(edge)

                ## let p be the fraction of entries in isClosed set to true. 
                ## ??? This serves as an estimate of Triangles per Wedge (T/W) ???
                p = this.isClosed.sum()/len(this.isClosed)

                ## K, the transitivity is equal to 3T/W, hence obtained through 3 * p
                Kt = 3 * p

                ## In order to estimate T from K, we need to estimate the the total number of wedges W.
                # This is done by reverse engineering the birthday paradox: 
                Tt = (p * this.t**2 / se * (se-1) ) * this.tot_wedges
                
                print("t: {}, K: {}, T: {} at {}".format(this.t, Kt, Tt, ctime(time())))
                if (this.t % 100 == 0): 
                    print("t: {}, K: {}, T: {} at {}".format(this.t, Kt, Tt, ctime(time())))

                this.t += 1

            this.K = Kt
            this.T = Tt

            print("Finished after {} steps, K = {}, T = {} at {}".format(this.t, this.K, this.T, ctime(time())))

    def update(this, et): 
        new_edges = 0
        new_wedges = 0

        
        ## 1. Check if et creates any triangles
        for i, wedge in enumerate(this.wedge_res): 
            if this.isTriangle(wedge, et): ## if wedge_res[i] closed by et:
                print("Triangles found!")
                this.isClosed[i] = True
        
        
        
        ## 2. Randomly update the sampled edges 
        ### (this happens very infrequently, approx: sum(se/t) = se * ln(stream_length))
        for i in range(this.se):
            
            #pick a random number x in [0, 1]
            x = np.random.rand()
            
            if (x <= i/this.t): 
                this.edge_res[i] = et
                new_edges += 1
            
            
            
        ## 3. If any new edges has been added
        if new_edges:             
            
            # get all wedges formed by the edges in edge_res
            all_wedges = this.getAllWedges()
            
            # update tot_wedges: the number of wedges in this.edge_res
            this.tot_wedges = len(all_wedges)
            
            # determine Nt (wedges involving et) and let new_wedges = |Nt|
            Nt = this.getWedgesWithEdge(all_wedges, et)
            new_wedges = len(Nt)
            
            
            ## Randomly update the sampled wedges
            for i in range(this.sw): 
                #pick a random number x in [0, 1]
                x = np.random.rand()   
                
                if (x <= new_wedges/this.tot_wedges):
                    # pick uniform random w in Nt                    
                    this.wedge_res[i] = random.sample(Nt, 1)[0]
                    #print("replacing wedge_res")
                    #print(Nt)
                    #print(this.wedge_res[i])
                    this.isClosed[i] = False

                    
                    
    ## Support Methods ## 
        
    def isTriangle(this, wedge, edge): 
        ## determines if a wedge and an edge forms a triange   
        if (edge[0] == edge[1]): 
            print("Found stupid edge {}".format(edge))
            return False
        
        e1 = set(wedge[0])
        e2 = set(wedge[1])
        leaves = e1 ^ e2
        
        if ((edge[0] in leaves) and (edge[1] in leaves)):
            return True 
        else: 
            return False
        
    def getWedgesWithEdge(this, wedges, edge):
        wedges_with_edge = set()
        for wedge in wedges: 
            if (edge in wedge): 
                wedges_with_edge.add(wedge)
                
        return wedges_with_edge
                
    
    def getAllWedges(this): 
        ## returns all wedges formed by the edges in edge_res
        wedges = set()
        for e1 in this.edge_res: 
            for e2 in this.edge_res: 
                # A wedge is a tuple of edges with intersection size exactly one
                if len(set(e1) & set(e2)) == 1: 
                    wedges.add((e1, e2))
        return wedges 
    

In [44]:
## Hyperparams 
se = 2000
sw = 1000
early_stop = 100000

In [45]:


ST = StreamingTriangles(path, se, sw)



Starting at Sat Nov 24 17:03:50 2018
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles found!
Not triangles foun

KeyboardInterrupt: 

In [ ]:
## Tests


ST = StreamingTriangles(path, se, sw, 0)

wedge = ((0, 1), (1, 2))
trueEdge = (2, 0)
falseEdge = (2, 1)
stupidEdge = (1, 1)

print("{}: expected: True".format(ST.isTriangle(wedge, trueEdge)))

print("{}: expected: False".format(ST.isTriangle(wedge, falseEdge)))

print("{}: expected: False".format(ST.isTriangle(wedge, stupidEdge)))


wedges = set([((0, 1), (1, 2)), ((1, 2), (2, 3)), ((1, 3), (2, 3))])
falseEdge = (0, 2)
trueEdge = (1, 2)

print("{}: expected: [(](0, 1), (1, 2)), ((1, 2), (2, 3))]".format(ST.getWedgesWithEdge(wedges, trueEdge)))

print("{}: expected: empty set".format(ST.getWedgesWithEdge(wedges, falseEdge)))


